In [13]:
# Importation des bibliothèques nécessaires
import pandas as pd
from sklearn.model_selection import train_test_split,GridSearchCV,RandomizedSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, roc_curve, auc,accuracy_score
import matplotlib.pyplot as plt


In [3]:
# Chargement du jeu de données de séries temporelles : je choisi train.csv
data = pd.read_csv("../data/BenignAndMaliciousDataset.csv")

# Les colonnes à garder comme features
features = [
  'Domain', 'Ip', 'ASN', 
  'HttpResponseCode', 'SubdomainNumber', 'Entropy', 'EntropyOfSubDomains',
  'SpecialCharSequence', 'DomainLength'
]

In [4]:
# La colonne cible
target = 'Class'

# Séparation en features et target
X = data[features]  
y = data[target]

In [5]:
# Split train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [6]:
#  k-NN (k-Nearest Neighbors)

# On Choisie le nombre de voisins aleatoirement dans un premier temps  (k)
k_neighbors = 5

# Entraîner le modèle k-NN
knn = KNeighborsClassifier(n_neighbors=k_neighbors)
knn.fit(X_train, y_train)

# Prédire sur le jeu de test
y_pred = knn.predict(X_test)

print(y_pred)

#Evaluer la performance
print("Classification Report for k-NN:")
# print(classification_report(y_test, y_pred))
print(accuracy_score(y_pred,y_test))


[0 1 1 ... 1 0 1]
Classification Report for k-NN:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      8854
           1       1.00      1.00      1.00      9146

    accuracy                           1.00     18000
   macro avg       1.00      1.00      1.00     18000
weighted avg       1.00      1.00      1.00     18000



In [ ]:
##### affichage de la courbe de roc


# Courbe ROC pour k-NN
probs = knn.predict_proba(X_test)
preds_pos = probs[:, 1]
fpr, tpr, thresholds = roc_curve(y_test, preds_pos)

# Tracer la courbe ROC pour k-NN
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='k-NN')
plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
plt.xlabel('Faux positifs')
plt.ylabel('Vrais positifs')
plt.title('Courbe ROC pour k-NN')
plt.legend(loc="lower right")
plt.show()

# Calculer l'aire sous la courbe pour k-NN
roc_auc = auc(fpr, tpr)
print('AUC ROC for k-NN: {:.2f}'.format(roc_auc))


In [9]:
## Optimisation hyperparametre Hyperparametres

hyperparametres = { 'n_neighbors':range(1,10),'weights':['uniform','distance']}


grid = GridSearchCV(KNeighborsClassifier(),hyperparametres,cv = 10,scoring='accuracy')


grid.fit(X_train, y_train)

GridSearchCV(cv=10, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': range(1, 10),
                         'weights': ['uniform', 'distance']},
             scoring='accuracy')

In [10]:
#### affichage des meilleur hyperparamtre 

grid.best_params_

{'n_neighbors': 3, 'weights': 'distance'}

In [11]:
#### on fait les prediction sur les donne d'entrainement

modele = grid.best_estimator_

modele.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3, weights='distance')

In [14]:
#### prediction sur les donnees de test 

# Prédire sur le jeu de test
y_pred = modele.predict(X_test)

print(y_pred)

print(accuracy_score(y_pred,y_test))


[0 1 1 ... 1 0 1]
0.9975555555555555


In [11]:
# Sauvegarder le modèle entraîné
import pickle
pickle.dump(knn, open('knn_model.pkl', 'wb'))

# Charger le modèle
knn = pickle.load(open('knn_model.pkl', 'rb'))
